In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/OPML/20210807_Titanic/彥嘉

/content/drive/Shareddrives/OPML/Seminar/20210807_Titanic_廖家緯/彥嘉


In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 916 kB 4.9 MB/s 
     |████████████████████████████████| 829 kB 41.3 MB/s 
     |████████████████████████████████| 272 kB 55.3 MB/s 
     |████████████████████████████████| 118 kB 58.6 MB/s 
     |████████████████████████████████| 636 kB 42.2 MB/s 
     |████████████████████████████████| 1.3 MB 43.3 MB/s 
     |████████████████████████████████| 294 kB 71.8 MB/s 
     |████████████████████████████████| 142 kB 70.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=103bee0114237ba23fc7e729ef3112212f6b10746ea7483af27f45bbbabe7487
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchvision import transforms
from pytorch_lightning import LightningModule, LightningDataModule, Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from PIL import Image

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
data_dir = "/content/drive/Shareddrives/OPML/Seminar/20210814_Mango_identification/data/mango_1st_stage/"
data = pd.read_csv(data_dir+"train.csv")
data.head()

,image_id,label
0,00002.jpg,C
1,00003.jpg,C
2,00004.jpg,B
3,00005.jpg,A
4,00007.jpg,A


In [ ]:
labelencoder_X = LabelEncoder()

In [ ]:
data_list = data.image_id.tolist()
# data_list

In [ ]:
data_label = data.label.tolist()
data_label = labelencoder_X.fit_transform(data_label)
label = np.zeros((data_label.shape[0],3))
label[np.arange(data_label.size),data_label]=1
print(data_label[:5])
print(label[:5])


[2 2 1 0 0]
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [ ]:
data_list_rename = [data_dir+"train/"+i for i in data_list]

In [ ]:
class Transform():
  def __init__(self, keys):
    self.keys = keys
  
  def __call__(self, data):
    for key in self.keys:
        if key in data:
            pass    
        else:
            raise KeyError(f'{key} is not a key of {data}')
    
    return data

In [ ]:
class DataModule(LightningDataModule):
  def __init__(self):
    super(DataModule, self).__init__()
  
  def prepare_data(self):
    data = [{
            'image': data_list_rename[i],
            'label': label[i]
         } for i in range(len(data_list_rename)) ]
    train_file = data[:5000]
    val_file = data[5000:]

    class Load(Transform):
        def __init__(self, keys):
            self.keys = keys
  
        def __call__(self, data):
            for key in self.keys:
                if key in data:
                    data[key] = transforms.ToPILImage()(data[key])
                else:
                    raise KeyError(f'{key} is not a key of {data}')
            return data
    class Resize(Transform):
        def __init__(self, keys):
            self.keys = keys
  
        def __call__(self, data):
            for key in self.keys:
                if key in data:
                    data[key] = transforms.Resize(size=(64,64))(data[key])
                else:
                    raise KeyError(f'{key} is not a key of {data}')
            
            return data
    class Totensor(Transform):
        def __init__(self, keys):
            self.keys = keys
  
        def __call__(self, data):
            for key in self.keys:
                if key in data:
                    data[key] = transforms.PILToTensor()(data[key])
                else:
                    raise KeyError(f'{key} is not a key of {data}')
            
            return data
    class LTotensor(Transform):
        def __init__(self, keys):
            self.keys = keys
  
        def __call__(self, data):
            for key in self.keys:
                if key in data:
                    data[key] = transforms.ToTensor()(data[key])
                else:
                    raise KeyError(f'{key} is not a key of {data}')
            
            return data

    train_transforms = transforms.Compose([Load(keys=["image"]),Resize(keys=["image"]),Totensor(keys=["image"]),LTotensor(keys=["label"])])
    self.train_ds = Dataset(data=train_file, transform = train_transforms)
    self.val_ds = Dataset(data=val_file, transform = train_transforms)

  def train_dataloader(self):
    return DataLoader(self.train_ds, batch_size = 10, num_workers = 0)
  
  def val_dataloader(self):
    return DataLoader(self.val_ds, batch_size = 10, num_workers = 0)

In [ ]:
class Net(LightningModule):
    def __init__(self, lr):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=16, padding=1, kernel_size=3, stride=2)
        self.bn = nn.BatchNorm2d(16)
        self.prelu = nn.PReLU(),
        self.dropout = nn.Dropout(p=0.5)
        self.layer  = nn.Linear(16*10*10, 3)

        self.save_hyperparameters()
        self.criterion = nn.CrossEntropyLoss()

        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        x = self.dropout(x)
        x = self.layer(x)
        x = torch.sigmoid(x)
        
        return x
    
    
    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.hparams.lr)
        
        return optimizer
    
    
    def compute_loss(self, x, y):
        loss = self.criterion(x,y)
        
        return loss
    
    
    def compute_acc(self, x, y):        
        return acc
    
    
    def evaluate(self, batch):
        X, y   = batch
        output = self(X)
        loss   = self.compute_loss(output, y)
        acc    = self.compute_acc(output, y)
            
        return {'loss': loss, 'acc': acc}
        
        
    def training_step(self, batch, batch_idx):
        output = self.evaluate(batch)

        return output
    
    
    def validation_step(self, batch, batch_idx):
        output = self.evaluate(batch)
        
        return output
    
    
    def training_epoch_end(self, outputs):
        meanloss = torch.mean(torch.stack([o['loss'] for o in outputs]))
        meanacc = torch.mean(torch.stack([o['acc'] for o in outputs]))
        self.log('step', self.trainer.current_epoch)
        self.log('train_acc', meanacc, prog_bar=True)
        self.log('step', self.trainer.current_epoch)
        self.log('train_loss', meanloss, prog_bar=True)    
        
        return None
    
    
    def validation_epoch_end(self, outputs):
        meanloss = torch.mean(torch.stack([o['loss'] for o in outputs]))
        meanacc = torch.mean(torch.stack([o['acc'] for o in outputs]))
        self.log('step', self.trainer.current_epoch)
        self.log('val_acc', meanacc, prog_bar=True)
        self.log('step', self.trainer.current_epoch)
        self.log('val_loss', meanloss, prog_bar=True)    
        
        return None

In [ ]:
net = Net(lr=1e-3)
datamodule = DataModule()

In [ ]:
trainer = Trainer(
    max_epochs=10,
    gpus=0
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(net, datamodule=datamodule)

TypeError: ignored